In [1]:
library("dplyr")

Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



## Inputs: 

1. Set of nodes *Z*
2. Ordering of the nodes
3. Upper bound on the number of parents (for each node) *u*
4. Database *D* of m cases

In [18]:
N_tensor <- function(D, i, V.i, pi.i){
    result <- D %>% count(D[,pi.i], name='j_counts')
    if (length(pi.i) == 1){colnames(result)[1] <- 'j'}
        for (k in V.i){
            k_count <- D[D[, i]==k,] %>% count(D[D[, i]==k,pi.i])
            if (length(pi.i) == 1){colnames(k_count)[1] <- 'j'}
            colnames(k_count)[length(colnames(k_count))]<- k
            result <- merge(result, k_count, all.x=TRUE) 
        }
        result[is.na(result)] <- 0
    #names(result)[1] <- 'j_counts'
    #names(result)[2:(length(V.i)+1)] <- V.i
    cat('\n')
    print(result)
    return(result)
}

g <- function(V.i, r.i, N.tensor){
    N.ij <- N.tensor$j_counts
    q.i <- length(N.ij) # number of unique combinations of the parents realizations
    result <- 1
    for (j in 1:q.i) {
        a <- 1
        for (k in V.i){a <- a*factorial(N.tensor[j,as.character(k)])}
        result <- result*a*factorial(r.i-1)/factorial(N.ij[j]+r.i-1)
    }
    return(result)
}


k2 <- function(Z, u, D){
    V <- unname(sapply(D, unique)) #it's a matrix
    r <- unname(sapply(D, n_distinct)) #it's a vector
    parents <- list()
    for (i in 1:length(Z)){
        cat('\n iteration', i)
        pi.i <- NULL 
        Nijk <- N_tensor(D, Z[i], V[,i], pi.i)
        P_old <- g(V[,i],r[i], Nijk)
        proceed <- TRUE
        changed = FALSE
        while (proceed & (length(pi.i)<u & i>1)){
            pred.i <- setdiff(Z[1:i-1], pi.i)
            P <- 0
            for (node in pred.i){
                new_set <- union(pi.i, node)
                Nijk <- N_tensor(D, Z[i], V[,i], new_set)
                P <- g(V[,i], r[i], Nijk)
                if (P > P_old){
                    P_old <- P
                    temp.pi <- new_set
                    changed = TRUE
                }
            }
            if (changed != TRUE){
                proceed = FALSE
            } else{
                pi.i <- temp.pi
                changed = FALSE
            }
        }
        cat('\n P:', P_old)
        parents <- append(parents, list(pi.i))
    } 
    return(parents)
}


In [19]:
x1 <- c(1,1,0,1,0,0,1,0,1,0)
x2 <- c(0,1,0,1,0,1,1,0,1,0)
x3<- c(0,1,1,1,0,1,1,0,1,0)

D <- data.frame(x1, x2, x3)

print(D)

   x1 x2 x3
1   1  0  0
2   1  1  1
3   0  0  1
4   1  1  1
5   0  0  0
6   0  1  1
7   1  1  1
8   0  0  0
9   1  1  1
10  0  0  0


In [20]:
papitos <- k2(c(1,2,3), u=3, D)


 iteration 1
  j_counts 1 0
1       10 5 5

 P: 0.0003607504
 iteration 2
  j_counts 0 1
1       10 5 5

  help j_counts 0 1
1    0        5 4 1
2    1        5 1 4

 P: 0.001111111
 iteration 3
  j_counts 0 1
1       10 4 6

  help j_counts 0 1
1    0        5 3 2
2    1        5 1 4

  help j_counts 0 1
1    0        5 4 1
2    1        5 0 5

  x2 x1 j_counts 0 1
1  0  0        4 3 1
2  0  1        1 1 0
3  1  0        1 0 1
4  1  1        4 0 4

 P: 0.005555556

In [21]:
papitos

[[1]]
NULL

[[2]]
[1] 1

[[3]]
[1] 2

In [425]:
#pi.i <- c(2,3)
#D[,pi.i] %>% group_by_all() %>% count()
a <- D %>% count(D[,pi.i], name='j_count')
#a <- D %>% count(D[,1], D[,2],name='j_count')

In [429]:
#D[D[, 3]==0,pi.i] %>% group_by_all() %>% summarise(name=n())
pi.i<-c(1,2)
#D[D[, 3]==0, ] %>% count(D[D[, 3]==0,pi.i])
i=3
ss=0
x <- D[D[, i]==ss, ] %>% count(D[D[, i]==ss,pi.i])
#merge(D1, k_count, all.x=TRUE)
#result[result[, 3]==0, ] %>% count(x1, x2)
x
a
merge(a,x)

x1,x2,n
0,0,3
1,0,1


x1,x2,j_count
0,0,4
0,1,1
1,0,1
1,1,4


x1,x2,j_count,n
0,0,4,3
1,0,1,1
